In [1]:
from requests_oauthlib import OAuth1Session
from datetime import datetime as dt
import json, csv, time, random
from conf import *

"""
- this program is....
  - get all user's tweet_ids
  - export to csv(seqnum,tweet_content,postdate)
"""

# make session
twitter = OAuth1Session(CK, CS, t_AT, AS)

t_user_id="minyoruminyon"

In [2]:
all_twit = json.loads(
        twitter.get("https://api.twitter.com/1.1/users/lookup.json",
        params={'screen_name': t_user_id}).text
)[0]['statuses_count']

In [4]:
# start retrievin!
# API endpoint
url = "https://api.twitter.com/1.1/statuses/user_timeline.json"
# Num of tweets which this program retrieves at once
twit = 200
twit_cnt = 0
max_id = ""
# savefile
date = parsed_time = dt.today().strftime("%Y%m%d%H%M%S")
csvname = "CSV/" + t_user_id + "_all_tweet_" + str(date) + ".csv"
f = open(csvname, 'a')
# prepare for a savefile
writer = csv.writer(f)
header = ['tweet_id', 'posted_date', 'quoted_user',
          'qtweet_id', 'fav', 'rt', 'result', 'script_first']
writer.writerow(header)
# =============================================================
for try_cnt in range(1,10**19):
    print("Now:", try_cnt, end="\r")
    # make request json
    params = {
        'count' : twit,
        'screen_name' : t_user_id
    }
    # set previous id
    if try_cnt != 1:
        params['max_id'] = max_id
    # =========================================================
    # keep trying till getting a fine response 
    while True:
        req = twitter.get(url, params=params)
        print("%d:csv:" % req.status_code, try_cnt,"(%d" % twit_cnt,
              "/%d)" % all_twit,  end="\r")
        if req.status_code == 200 and len(json.loads(req.text)) != 0:
            # =========================================================
            # convert response into json
            timelines = json.loads(req.text)
            twit_cnt += len(timelines)
            """
            CSV format:
                <tweet id>, <postdate>, <quoted user>, <quoted id>,
                <fav>, <rt>, <result>, <script first>
            """
            for line in timelines:
                if not 'quoted_status' in line:
                    line['quoted_status'] = {'text':'',
                                             'user':{'screen_name':''}}
                    line['quoted_status_id'] = ''
                parsed_time = dt.strptime(
                    line['created_at'],'%a %b %d %H:%M:%S %z %Y')
                writer.writerow([
                        line['id'],
                        parsed_time.strftime("%Y/%m/%d %H:%M:%S"),
                        line['quoted_status']['user']['screen_name'],
                        line['quoted_status_id'],
                        line['favorite_count'],
                        line['retweet_count'],
                        line['text'],
                        line['quoted_status']['text'],
                ])
                max_id = int(line['id']) - 1
            time.sleep(3 + random.random())
            break # if writing csv is successful
            # =========================================================
        elif req.status_code == 429:
            print("Error: 429", " " * 30)
            print("Now:", try_cnt, ":oldest_date:", parsed_time.strftime("%Y/%m/%d %H:%M:%S"))
            for w in range(0,900):
                print("%d/(900)" % w, end="\r")
                time.sleep(1) # 15 min wait for submitting a new req.
                                # Because Twitter API limits freq. of request
                                # 180 req. / 15 min.
        else:
            print("Error: %d" % req.status_code, " " * 30)
            print("Now:", try_cnt, ":oldest_date:", parsed_time.strftime("%Y/%m/%d %H:%M:%S"))
            for w in range(0,900):
                print("%d/(900)" % w, end="\r")
                time.sleep(1) # 15 min wait for submitting a new req.
                                # Because Twitter API limits freq. of request
                                # 180 req. / 15 min.

f.close()
print("===DONE===")

Error: 200          7119)
Now: 18 :oldest_date: 2020/01/31 13:23:39


KeyboardInterrupt: 